In [7]:
import pandas
import pandas as pd
import numpy as np
import sys
import sklearn
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tqdm import tqdm

deepctr_path = '/root/linghui/rec-fairness/'
sys.path.append(deepctr_path)
import deepctr
from deepctr.models import *
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, get_feature_names
# from utilis import get_feat_dict

def get_feat_dict(data):
    if data == 'ml-1m':
        feat_dict = {'movie_id': 3706,
                 'user_id': 6040,
                 'gender': 2,
                 'age': 7,
                 'occupation': 21,
                 'zipcode': 3439}
    if data == 'black-friday':
        feat_dict = {'user_id': 5891,
                 'product_id': 3631,
                 'gender': 2,
                 'age': 7,
                 'occupation': 21,
                 'city': 3,
                 'stay_years': 5,
                 'martial_status': 2,
                 'cate_1': 20,
                 'cate_2': 18,
                 'cate_3': 16,
                 'price': 23961 }
    return feat_dict

2022-07-14 10:09:58.882903: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
DeepCTR version 0.9.1 detected. Your version is 0.9.0.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.9.1


In [8]:
#load train_data  ml-1m

saved_data_path = '/data/linghui/ml-1m/processed_data/'
# train_data_path = saved_data_path + 'train_data.csv' 
# test_data_path = saved_data_path + 'test_data.csv'
train_data_path = saved_data_path + 'train_data_liuyi.csv' 
test_data_path = saved_data_path + 'test_data_liuyi.csv'

train_data = pd.read_csv(train_data_path,index_col=0)

/opt/conda/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
train_data

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zipcode
0,0,2969,1,978300019,"Face in the Crowd, A (1957)",Drama,0,0,10,1588
1,0,1178,1,978300055,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War,0,0,10,1588
2,0,1574,1,978300055,"Peacemaker, The (1997)",Action|Thriller|War,0,0,10,1588
3,0,957,1,978300055,"African Queen, The (1951)",Action|Adventure|Romance|War,0,0,10,1588
4,0,2147,1,978300103,"Skin Game, The (1931)",Drama,0,0,10,1588
...,...,...,...,...,...,...,...,...,...,...
1988333,6039,1074,0,0,Platoon (1986),Drama|War,1,2,6,466
1988334,6039,627,0,0,Land and Freedom (Tierra y libertad) (1995),War,1,2,6,466
1988335,6039,2882,0,0,"Fistful of Dollars, A (1964)",Action|Western,1,2,6,466
1988336,6039,3356,0,0,Mo' Better Blues (1990),Drama,1,2,6,466


In [ ]:
#process genres columns (optional)

from ast import literal_eval

genres_list = train_data['genres'].tolist()
genres = []
for i in range(len(genres_list)):
    genres.append(literal_eval(genres_list[i]))
train_data['genres'] = genres


In [9]:
#get fixlen_feature_columns and model input

dataset = 'ml-1m'
feat_dict = get_feat_dict(dataset)

sparse_features = ["movie_id", "user_id",
                   "gender", "age", "occupation", "zipcode", ]

fixlen_feature_columns = [SparseFeat(feat, feat_dict[feat], embedding_dim=4)
								  for feat in sparse_features]

movie_size = 18
max_len = 6

train_feats = ["movie_id", "user_id",
                   "gender", "age", "occupation", "zipcode"]

fixlen_feature_columns = [SparseFeat(feat, feat_dict[feat], embedding_dim=4)
								  for feat in train_feats]

# use_weighted_sequence = False
# if use_weighted_sequence:
#     varlen_feature_columns = [VarLenSparseFeat(SparseFeat('genres', vocabulary_size = movie_size + 1, 
#                                                           embedding_dim=4), maxlen=max_len, combiner='mean', weight_name='genres_weight')]  
#     # Notice : value 0 is for padding for sequence input feature
# else:
#     varlen_feature_columns = [VarLenSparseFeat(SparseFeat('genres', vocabulary_size = movie_size + 1, 
#                                                           embedding_dim=4), maxlen=max_len, combiner='mean', weight_name=None)]  
    # Notice : value 0 is for padding for sequence input feature

# linear_feature_columns = fixlen_feature_columns + varlen_feature_columns
# dnn_feature_columns = fixlen_feature_columns + varlen_feature_columns

linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

train_data = sklearn.utils.shuffle(train_data)

train_model_input = {name: train_data[name].values for name in train_feats}
label = train_data['rating'].values

In [13]:
def train_ml_ctr_model(model_name,batch_size, epochs,validation_split=0.2):
    if model_name == 'deepfm':
        model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
    if model_name == 'afm':
        model = AFM(linear_feature_columns, dnn_feature_columns, task='binary')
    if model_name == 'wdl':
        model = WDL(linear_feature_columns, dnn_feature_columns, task='binary')
    if model_name == 'nfm':
        model = NFM(linear_feature_columns, dnn_feature_columns, task='binary')
    saved_model_path = '/data/linghui/saved_model/' + model_name + '-ml-1m/'
    check_path = saved_model_path + model_name + '-ml-1m-liuyi.ckpt'
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy',
              metrics=['AUC', 'Precision', 'Recall'])
    model.summary()
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint(check_path, save_weights_only=True,
                                                 save_best_only=True,
                                                 monitor='val_loss')
    print('begain training')
    model.fit(train_model_input, label,
						batch_size=batch_size, epochs=epochs, verbose=2,
                        callbacks=[checkpoint],
						validation_split=validation_split)